In [1]:
import MDAnalysis as md
import numpy as np
import tensorflow as tf
#from nmrdata.nmrdata.parse import parse_universe
#from nmrdata.nmrdata import loading
import nmrdata
import pickle
from simtk.openmm import app
import math
import os

In [2]:
def pdb_to_records(path, embedding_dicts, max_atoms=1024):
	#load pdb
    pdb = app.PDBFile(path)
    topo = pdb.topology
    NN = 16
    atoms = np.zeros((max_atoms), dtype=np.int64) # atom type
	#mask = np.zeros((max_atoms), dtype=np.float)
    pos = pdb.getPositions(True) # positions
    print(len(pos))
    position = np.zeros((max_atoms-len(pos), 3), dtype=np.float)
	#peaks = np.zeros((max_atoms), dtype=np.float)
    names = np.zeros((max_atoms), dtype=np.int64)
    nlist = np.zeros((max_atoms, NN, 3), dtype=np.float)
    index = 0
    A = np.zeros((max_atoms, max_atoms), dtype=np.float)
    for residue in topo.residues():
        resname = residue.name
        for bond in residue.bonds():
            A[bond.atom1.index,bond.atom2.index] = 1
            A[bond.atom2.index, bond.atom1.index] = 1
    for atom in topo.atoms():
        atom_name = atom.residue.name + '-' + atom.name
        if atom_name not in embedding_dicts['name']:
            embedding_dicts['name'][atom_name] = len(embedding_dicts['name'])
        names[atom.index] = embedding_dicts['name'][atom_name]
		#mask[atom.index]=float(atom.element.symbol=='H')
        atoms[atom.index] = embedding_dicts['atom'][atom.element.symbol]
    positions =  np.concatenate((pos, position), axis=0)
    frame_nlist = nlist_tf_model(positions, NN)
    for i in range(frame_nlist.shape[0]):
        for j in range(NN):
            nlist[i,j,1] = frame_nlist[i,j,1]
            nlist[i,j,0] = frame_nlist[i,j,0]
            N_index = int(frame_nlist[i,j,1].numpy())
            if A[i,N_index] == 0:
                nlist[i,j,2] = embedding_dicts['nlist']['nonbonded']
            else:
                nlist[i,j,2] = embedding_dicts['nlist'][1]
    return nlist

In [7]:
loading.load_embeddings('../nmrdata/nmrdata/data/embeddings.pb')

{'atom': {'X': 0, 'Z': 1},
 'bond': {'none': 0, 'single': 1, 'double': 2, 'triple': 3, 'aromatic': 4},
 'exp': {'': 0},
 'class': {'ALA': 0,
  'ARG': 1,
  'ASN': 2,
  'ASP': 3,
  'CYS': 4,
  'GLU': 5,
  'GLN': 6,
  'GLY': 7,
  'HIS': 8,
  'ILE': 9,
  'LEU': 10,
  'LYS': 11,
  'MET': 12,
  'PHE': 13,
  'PRO': 14,
  'SER': 15,
  'THR': 16,
  'TRP': 17,
  'TYR': 18,
  'VAL': 19,
  'MB': 20},
 'nlist': {'none': 0, 'nonbonded': 1},
 'name': {'X': 0}}

In [3]:
CGpdb = '../mtn/1A2P_CG.pdb'
neighbor_num = 16
embedding_path = './embeddings.pb'
PDBFile = '../shiftx2-trainset-June2011/PDB-training/R001_1QRXA.pdb'
u = md.Universe(CGpdb)

#A = parse_universe(u, neighbor_num)
A = pdb_to_records(CGpdb, loading.load_embeddings(embedding_path))
#print(u.atoms)
print(A)

744


KeyError: 'B'

In [2]:
CGpdb = '../mtn/1A2P_CG.pdb'
u = md.Universe(CGpdb)

In [18]:
CGDict = {}
CGDict['VAL'] = ['P5','AC2']
CGDict['ILE'] = ['P5','AC1']
CGDict['ASN'] = ['P5','P5']
CGDict['THR'] = ['P5','P1']
CGDict['PHE'] = ['P5','SC5','SC5','SC5']
CGDict['ASP'] = ['P5','Qa'] 
CGDict['GLY'] = ['P5']
CGDict['ALA'] = ['P4']
CGDict['TYR'] = ['P5','SC4','SC4','SP1']
CGDict['LEU'] = ['P5','AC1'] 
CGDict['GLN'] = ['P5','P4']
CGDict['HIS'] = ['P5','SC4','SP1','SP1']
CGDict['LYS'] = ['P5','C3','Qd']
CGDict['PRO'] = ['P4','C3']
CGDict['ASP'] = ['P5','Qa']
CGDict['SER'] = ['P5','P1']
CGDict['GLU'] = ['P5','Qa'] 
CGDict['TRP'] = ['P5','SC4','SNd','SC5','SC5']
CGDict['ARG'] = ['P5','N0','Qd']
CGDict['THR'] = ['P5','P1','P5','P1']

In [19]:
CGDict

{'VAL': ['P5', 'AC2'],
 'ILE': ['P5', 'AC1'],
 'ASN': ['P5', 'P5'],
 'THR': ['P5', 'P1', 'P5', 'P1'],
 'PHE': ['P5', 'SC5', 'SC5', 'SC5'],
 'ASP': ['P5', 'Qa'],
 'GLY': ['P5'],
 'ALA': ['P4'],
 'TYR': ['P5', 'SC4', 'SC4', 'SP1'],
 'LEU': ['P5', 'AC1'],
 'GLN': ['P5', 'P4'],
 'HIS': ['P5', 'SC4', 'SP1', 'SP1'],
 'LYS': ['P5', 'C3', 'Qd'],
 'PRO': ['P4', 'C3'],
 'SER': ['P5', 'P1'],
 'GLU': ['P5', 'Qa'],
 'TRP': ['P5', 'SC4', 'SNd', 'SC5', 'SC5'],
 'ARG': ['P5', 'N0', 'Qd']}

# 不急 做自己

In [29]:
previous = u.atoms[-1]
count = 0
for atom in u.atoms:
    residue = atom.resname
    resid = atom.resid
    if residue == previous.resname and resid == previous.resid:
        count += 1
    else: count = 0
    #print(residue)
    print(CGDict[residue][count])
    atom.name = CGDict[residue][count]
    previous = atom
    

P5
AC2
P5
AC1
P5
P5
P5
P1
P5
SC5
SC5
SC5
P5
Qa
P5
P5
AC2
P4
P5
Qa
P5
SC4
SC4
SP1
P5
AC1
P5
P4
P5
P1
P5
SC4
SC4
SP1
P5
SC4
SP1
SP1
P5
C3
Qd
P5
AC1
P4
C3
P5
Qa
P5
P5
P5
SC4
SC4
SP1
P5
AC1
P5
P1
P5
C3
Qd
P5
P1
P5
Qa
P4
P5
P4
P4
P5
AC1
P5
P5
SC4
SNd
SC5
SC5
P5
AC2
P4
P5
P1
P5
C3
Qd
P5
P5
P5
P5
AC1
P4
P5
Qa
P5
AC2
P4
P4
C3
P5
P5
C3
Qd
P5
P1
P5
AC1
P5
P5
P5
Qa
P5
AC1
P5
SC5
SC5
SC5
P5
P1
P5
P5
P5
N0
Qd
P5
Qa
P5
P5
C3
Qd
P5
AC1
P4
C3
P5
P5
C3
Qd
P5
P1
P5
P5
N0
Qd
P5
P1
P5
SC4
SNd
SC5
SC5
P5
N0
Qd
P5
Qa
P4
P5
Qa
P5
AC1
P5
P5
P5
SC4
SC4
SP1
P5
P1
P5
P1
P5
P5
SC5
SC5
SC5
P5
N0
Qd
P5
P5
P5
P1
P5
Qa
P5
N0
Qd
P5
AC1
P5
AC1
P5
SC4
SC4
SP1
P5
P1
P5
P1
P5
Qa
P5
SC4
SNd
SC5
SC5
P5
AC1
P5
AC1
P5
SC4
SC4
SP1
P5
C3
Qd
P5
P1
P5
P1
P5
Qa
P5
SC4
SP1
SP1
P5
SC4
SC4
SP1
P5
P4
P5
P1
P5
SC5
SC5
SC5
P5
P1
P5
C3
Qd
P5
AC1
P5
N0
Qd
P5
AC2
P5
AC1
P5
P5
P5
P1
P5
SC5
SC5
SC5
P5
Qa
P5
P5
AC2
P4
P5
Qa
P5
SC4
SC4
SP1
P5
AC1
P5
P4
P5
P1
P5
SC4
SC4
SP1
P5
SC4
SP1
SP1
P5
C3
Qd
P5
AC1
P4
C3
P5
Qa
P5
P5
P5
SC4
SC4
SP1
P5
AC1


In [30]:
u.atoms.names

array(['P5', 'AC2', 'P5', 'AC1', 'P5', 'P5', 'P5', 'P1', 'P5', 'SC5',
       'SC5', 'SC5', 'P5', 'Qa', 'P5', 'P5', 'AC2', 'P4', 'P5', 'Qa',
       'P5', 'SC4', 'SC4', 'SP1', 'P5', 'AC1', 'P5', 'P4', 'P5', 'P1',
       'P5', 'SC4', 'SC4', 'SP1', 'P5', 'SC4', 'SP1', 'SP1', 'P5', 'C3',
       'Qd', 'P5', 'AC1', 'P4', 'C3', 'P5', 'Qa', 'P5', 'P5', 'P5', 'SC4',
       'SC4', 'SP1', 'P5', 'AC1', 'P5', 'P1', 'P5', 'C3', 'Qd', 'P5',
       'P1', 'P5', 'Qa', 'P4', 'P5', 'P4', 'P4', 'P5', 'AC1', 'P5', 'P5',
       'SC4', 'SNd', 'SC5', 'SC5', 'P5', 'AC2', 'P4', 'P5', 'P1', 'P5',
       'C3', 'Qd', 'P5', 'P5', 'P5', 'P5', 'AC1', 'P4', 'P5', 'Qa', 'P5',
       'AC2', 'P4', 'P4', 'C3', 'P5', 'P5', 'C3', 'Qd', 'P5', 'P1', 'P5',
       'AC1', 'P5', 'P5', 'P5', 'Qa', 'P5', 'AC1', 'P5', 'SC5', 'SC5',
       'SC5', 'P5', 'P1', 'P5', 'P5', 'P5', 'N0', 'Qd', 'P5', 'Qa', 'P5',
       'P5', 'C3', 'Qd', 'P5', 'AC1', 'P4', 'C3', 'P5', 'P5', 'C3', 'Qd',
       'P5', 'P1', 'P5', 'P5', 'N0', 'Qd', 'P5', 'P1', 'P5